In [1]:
%load_ext autoreload
%autoreload 2

%cd ../../
!hostname

/p/fastdata/pli/Private/oberstrass1/datasets/vervet1818-3d
jrlogin06.jureca


In [2]:
import numpy as np
import SimpleITK as sitk

from vervet1818_3d.utils.transforms import downscale_image, index2coord, warp_coord, coord2ix

In [3]:
field_path = "in/volume-reconstruction/SpatialTransformation/Vervet1818aa_60mu_70ms_s1083_x00-14_y00-11_SpatialTransformation.nii"

###

field_image = sitk.ReadImage(field_path)

In [4]:
downscale = 32
interpolator = sitk.sitkLinear

##

small_image = downscale_image(field_image, downscale, interpolator)

In [5]:
small_transform = sitk.DisplacementFieldTransform(small_image)
field_transform = sitk.DisplacementFieldTransform(field_image)

In [6]:
n_samples = 10000
np.random.seed(299792458)

###

size = field_transform.GetDisplacementField().GetSize()

import numpy as np

indices = np.vstack((np.random.randint(0, size[1], n_samples), np.random.randint(0, size[0], n_samples))).T

In [7]:

diffs = []

for ix in indices:
    coord = index2coord(ix, (0.0013, 0.0013))

    coord1 = np.array(warp_coord(coord, small_transform))
    coord2 = np.array(warp_coord(coord, field_transform))

    d = np.linalg.norm(coord1 - coord2) / 0.0013

    if d > 1.0:
        print(ix)

    diffs.append(d)

In [8]:
np.mean(diffs)

0.026123053686007206

In [9]:
sorted(diffs)[::-1][:10]

[0.6965871425135431,
 0.6868458393909448,
 0.683593034368585,
 0.6579068439576261,
 0.6354960384089935,
 0.60044984981419,
 0.5969424180505178,
 0.5744582696596556,
 0.5661701573409578,
 0.5475352253647283]